# EURO 2024 ODDS:

In this notebook a simulation for the european football cup is presented, based on given power ratings for the teams and the group they belong. The groups draw has been already done so the real draw was used. Regarding the power ratings of the teams I am not sharing here how I derive them and probably a notebook explaining power rating calculations will follow.

When it comes to international football were there is lack of data, few games within curtain time periods, games without motivation (such as friendlies or very easy games) and players performing different compared to their clubs the process of creating power rating is less accurate and straightforward process compared to clubs' case. Some variables you have to include is national's teams ELO (or simple their results in recent matches), their market value (althought it needs adjustment based on player's age and league), the players' form with their clubs and least but most important the MARKET! Always the market, since betting in exchanges are free markets and such tournaments have huge liquidity, offers the most important information aand have to be prioritized. Although there are cases when the market is not accurate it can provide always a reality check for any model is based purely on data.

**If you have a different estimation for a team's power rating feel free to change it in the table named df and check that scenarios' odds (run the notebook in the order it is presented).

In [1]:
import numpy as np
import random
import pandas as pd
import itertools
from scipy.stats import poisson

#!pip install openpyxl

First the table with the teams and their groups and their ratings have to be read.

In [2]:
df = pd.read_excel('Ratings.xlsx')
df

,Team,Group,Ratings
0,Germany,A,-0.115000
1,Scotland,A,-1.333944
2,Hungary,A,-1.166863
3,Switzerland,A,-1.055169
4,Spain,B,-0.096620
5,Croatia,B,-0.190000
6,Italy,B,-0.220000
7,Albania,B,-1.782317
8,Slovenia,C,-1.508252
9,Denmark,C,-1.009134


As it was said above in international footbal it is difficult to model teams' power. Normally an attacking rating and a defensive rating would be needed, but in our case it has been observed that it brings more noise than accuracy. Based on previous tournaments the total goals' of an event can be calculted based entirely on teams difference of rating as you can see below. 

Football is following the poisson distribution and thus the goals a team will score can be calculated based on their goal expectancy used as mean in poisson didtribution. Knowing the goal difference between teams and their sum of goals each team's goal expectation can be found and simulate scores based on that.

In [3]:
def goals(adv):
    
    '''This function returns the total goal of a football event based on the teams' ratings difference. This 
        formula is based on historical data of World and European football Cups.'''
    
    return 0.2523*adv**2+2.6

def game_result(team_A_rating, team_B_rating):
    
    '''This function simualates a football game based on given teams' power ratings and returns the goals' scored
        from the home and away team.'''
    
    total_goals = goals(team_A_rating-team_B_rating)
    goals_A = (total_goals-(team_A_rating-team_B_rating))/2
    goals_H = total_goals - goals_A
    Goals_scored_H = poisson.rvs(goals_H, size=1)
    Goals_scored_A = poisson.rvs(goals_A, size=1)

    return Goals_scored_H[0], Goals_scored_A[0]

game_result(-1.34,-0.2)

(4, 2)

For the knockout stage of the tournament another esult function will be needed since there is the possibility of extra time and penalties. In the extra time the goal expectancy of the teams is expected to be analogous to their initial goal expectancy, to be accurate 1/3 due to the extra time to be 1/3 of the regular time of the game. For penalties a unifrom 50-50 chance is assumed for the teams (higher level of accuracy would have a small impactin the overall odds). 

In [4]:
def game_result_knockout(team_A_rating, team_B_rating):
    
    '''This function calculates the goals scored of the home and away team and the team which qualified (output)
    based on the teams' ratings (input).'''
    
    total_goals = goals(team_A_rating-team_B_rating)
    goals_A = (total_goals-(team_A_rating-team_B_rating))/2
    goals_H = total_goals - goals_A
    Goals_scored_H = poisson.rvs(goals_H, size=1)[0]
    Goals_scored_A = poisson.rvs(goals_A, size=1)[0]
    Goals_scored_H_extra = 0
    Goals_scored_A_extra = 0
    
    if Goals_scored_H == Goals_scored_A:
        Goals_scored_H_extra = poisson.rvs(goals_H/3, size=1)[0]
        Goals_scored_A_extra = poisson.rvs(goals_A/3, size=1)[0]
        
        if Goals_scored_H_extra == Goals_scored_A_extra:
            qual = int(np.round(random.random(),0))
        elif Goals_scored_H_extra > Goals_scored_A_extra:
            qual = 0
        else:
            qual = 1
        
    elif Goals_scored_H > Goals_scored_A:
        qual = 0
    else:
        qual = 1

    Goals_scored_H += Goals_scored_H_extra
    Goals_scored_A += Goals_scored_A_extra
    
    return Goals_scored_H, Goals_scored_A,qual



In [91]:
def Group_Sim_2(df = df):
    
    '''This function simulates the group stage of the european cup based on a given table, which includes teams'
        names and their group and their power ratings. It returns a dataframe icluding teams' points, goals and
        goals' difference for each team in the group stage and another dataframe including all the simulated scores
        in order to be used for tie break cases.'''
    
    array = np.zeros((len(df),len(df))) 
    index_values = df['Team']
    column_values = df['Team']
    df_tie_break_g_tot = pd.DataFrame(data = array, index = index_values, columns = column_values) 
    
    points = {}
    team_H_group_goals = {}
    team_diff_group_goals = {}
    points_single_sim = {}
    goals_h_single_sim = {}
    goals_a_single_sim = {}
    goals_h_diff_single_sim = {}
    goals_a_diff_single_sim = {}
    teams = df['Team']
    
    for i in range(0,len(teams)):
        points[teams[i]] = 0
        team_H_group_goals[teams[i]] = 0
        team_diff_group_goals[teams[i]] = 0
    
    Groups = np.unique(df["Group"])
    for i in Groups:   
        teams_names = df[df["Group"]==i]["Team"]
        games = list(itertools.combinations(teams_names, 2))
        for j in teams_names:
            points_single_sim[j] = 0
            goals_h_single_sim[j] = 0
            goals_a_single_sim[j] = 0
            goals_h_diff_single_sim[j] = 0
            goals_a_diff_single_sim[j] = 0
        
        for j in games:
            team_A_rating = df[df["Team"]==j[0]]["Ratings"].values[0]
            team_B_rating = df[df["Team"]==j[1]]["Ratings"].values[0]
            
            home_G, away_G = game_result(team_A_rating,team_B_rating)
            if home_G > away_G :
                points_single_sim[j[0]] += 3
            elif home_G < away_G :
                points_single_sim[j[1]] += 3
            else:
                points_single_sim[j[0]] += 1
                points_single_sim[j[1]] += 1
            
            df_tie_break_g_tot[j[0]][j[1]] = home_G
            df_tie_break_g_tot[j[1]][j[0]] = away_G
                
            goals_h_single_sim[j[0]] += home_G
            goals_a_single_sim[j[1]] += away_G
            goals_h_diff_single_sim[j[0]] += home_G - away_G
            goals_a_diff_single_sim[j[1]] += away_G - home_G
        
        for i in teams_names:
            points[i] = points_single_sim[i]
            team_H_group_goals[i] = goals_h_single_sim[i]+goals_a_single_sim[i]
            team_diff_group_goals[i] = goals_h_diff_single_sim[i]+goals_a_diff_single_sim[i]
    

    
    a = pd.DataFrame(list(points.items()), columns=['Team', 'Points'])
    b = pd.DataFrame(list(team_H_group_goals.items()), columns=['Team', 'Goals'])
    c = pd.DataFrame(list(team_diff_group_goals.items()), columns=['Team', 'Goals_diff'])
    
    group_summary = pd.merge(a,b, on='Team')
    group_summary = pd.merge(group_summary,c, on='Team')
    group_summary = pd.merge(group_summary,df[['Team','Group']],on='Team')
    
    
    return group_summary, df_tie_break_g_tot

a,b = Group_Sim_2()
a

,Team,Points,Goals,Goals_diff,Group
0,Germany,9,9,7,A
1,Scotland,6,6,1,A
2,Hungary,3,2,-3,A
3,Switzerland,0,4,-5,A
4,Spain,4,4,1,B
5,Croatia,7,3,2,B
6,Italy,5,3,1,B
7,Albania,0,1,-4,B
8,Slovenia,1,3,-4,C
9,Denmark,7,5,3,C


In [6]:
def group_first(bb ,tie_break):
    '''This function takes as inpute the resulting simulated group stage results for a given group (bb) and the 
       simulated scored table (tie_break) and returns the position of each team based on UEFA's tie break ruels.
       The main tie break rules are points in between equal teams, their goal difference and then their goals' 
       scored. If teams with same points are still equall their goal difference and overall goals (in the mentioned 
       order) count.'''

    bb = bb.sort_values('Points')
    goal_diff = {}
    resu_p = {}
    names = []
    
    for i in range(0,4):
        goal_diff[bb['Team'].iloc[3-i]] = 0
        resu_p[bb['Team'].iloc[3-i]] = 0
        names.append(bb['Team'].iloc[i])
        
    k = 0
    while k<4:
        no_equals = (bb['Points'] == bb.iloc[3-k]['Points']).sum()
        
        if no_equals > 1 and no_equals < 4:
            for i in range(k,k+no_equals):
                for j in range(k,k+no_equals):
                    if i!=j:
                        resu_p[names[3-i]] += ((tie_break[names[3-i]][names[3-j]] - tie_break[names[3-j]][names[3-i]])>0)*3
                        resu_p[names[3-i]] += ((tie_break[names[3-i]][names[3-j]] - tie_break[names[3-j]][names[3-i]]) == 0)
                        goal_diff[names[3-i]] += tie_break[names[3-i]][names[3-j]] - tie_break[names[3-j]][names[3-i]]

        k += no_equals
        
    a = pd.DataFrame(list(resu_p.items()), columns=['Team', 'resu_p'])
    b = pd.DataFrame(list(goal_diff.items()), columns=['Team', 'goal_diff_tie_break'])
    group_summary = pd.merge(a,b, on='Team')
    group_summary = pd.merge(bb,group_summary, on='Team')
    group_summary = group_summary.sort_values(['Points','resu_p','goal_diff_tie_break', 'Goals', 'Goals_diff'])
    return group_summary['Team']
    
    

In [7]:
def match_ups(all_third):
    
    '''This function recieves as input the groups with best thrid team (alphabetically ordered) and returns the 
        knockout brackets for these teams based on the EURO 2024 draw rules : 
        https://en.wikipedia.org/wiki/UEFA_Euro_2024  .'''
    
    scenarios = [['A','B','C','D'],['A','B','C','E'],['A','B','C','F'],
             ['A','B','D','E'],['A','B','D','F'],['A','B','E','F'],
             ['A','C','D','E'],['A','C','D','F'],['A','C','E','F'],
             ['A','D','E','F'],['B','C','D','E'],['B','C','D','F'],
             ['B','C','E','F'],['B','D','E','F'],['C','D','E','F']]
    
    B_1 = ['A','A','A','D','D','E','E','F','E','E','E','F','F','F','F']
    C_1 = ['D','E','F','E','F','F','D','D','F','F','D','D','E','E','E']
    E_1 = ['B','B','B','A','A','B','C','C','C','D','B','C','C','D','D']
    F_1 = ['C','C','C','B','B','A','A','A','A','A','C','B','B','B','C']
    
    for i in range(0,len(scenarios)):
        if scenarios[i] == all_third:
            break

    return [B_1[i],C_1[i],E_1[i],F_1[i]]

In [8]:
def knock_out_phase(knock_out,best_third,df = df):
    
    '''This function simulates the knock-out phase of the tournament based on the power rating's table, the teams
        which were qualified as top 2 (knock_out list) and the best third teams best_third . It returns a list of 
        names for each stage with the teams they managed to reach that stage (top 16,quarter final, semis,
        final, winner)'''

    knock_out_seed = []
    for i in range(0,16):
        knock_out_seed.append("0")
    knock_out_seed[0] = knock_out[2] 
    knock_out_seed[2] = knock_out[0]
    knock_out_seed[3] = knock_out[5]
    knock_out_seed[4] = knock_out[10]
    knock_out_seed[6] = knock_out[7]
    knock_out_seed[7] = knock_out[9]
    knock_out_seed[8] = knock_out[8]
    knock_out_seed[10] = knock_out[6]
    knock_out_seed[11] = knock_out[11]
    knock_out_seed[12] = knock_out[4]
    knock_out_seed[14] = knock_out[1]
    knock_out_seed[15] = knock_out[3]
    
    matchup_list = match_ups(list(best_third['Group']))
    k = 1
    for i in matchup_list:
        knock_out_seed[k] = best_third[best_third["Group"] == i]['Team'].values[0]
        k +=4
    
    
    quarter_f = []
    for i in range(0,16,2):
        team_A_rating = df[df["Team"]==knock_out_seed[i]]["Ratings"].values[0]
        team_B_rating = df[df["Team"]==knock_out_seed[i+1]]["Ratings"].values[0]
        goal_H, goal_A , qual = game_result_knockout(team_A_rating, team_B_rating)
        quarter_f.append(knock_out_seed[i+qual])
        
    semis = []
    for i in range(0,8,2):
        team_A_rating = df[df["Team"]==quarter_f[i]]["Ratings"].values[0]
        team_B_rating = df[df["Team"]==quarter_f[i+1]]["Ratings"].values[0]
        goal_H, goal_A , qual = game_result_knockout(team_A_rating, team_B_rating)
        semis.append(quarter_f[i+qual])
        
    final = []
    for i in range(0,4,2):
        team_A_rating = df[df["Team"]==semis[i]]["Ratings"].values[0]
        team_B_rating = df[df["Team"]==semis[i+1]]["Ratings"].values[0]
        goal_H, goal_A , qual = game_result_knockout(team_A_rating, team_B_rating)
        final.append(semis[i+qual])
        
    team_A_rating = df[df["Team"]==final[0]]["Ratings"].values[0]
    team_B_rating = df[df["Team"]==final[1]]["Ratings"].values[0]
    goal_H, goal_A , qual = game_result_knockout(team_A_rating, team_B_rating)
    winner = final[qual]
    
    return winner,final,semis,quarter_f,knock_out_seed

In [92]:
def All_Sim(N_sim = 1000 ,df =df):
    
    '''THis function has as input the number of simulations and a pd dataframe which includes the teams,
        the group they belong and their ratings. The output of this function are two dataframes, one which includes
        the odds of the team to reach the different stages of the tournament (final, quarter final, winner etc) and 
        another one with the odds of finishing positions in the group stage with the expected points and goals at
        that stage.'''
    
    to_win = {}
    to_reach_final = {}
    to_reach_semis = {}
    to_reach_qurter = {}
    to_qualify_group = {}
    
    points = {}
    goals_group = {}
    position_1 = {}
    position_2 = {}
    position_3 = {}
    position_4 = {}
    Groups = np.unique(df['Group'].values)
    
    
    teams_names = df["Team"]
    position = np.zeros((4,len(teams_names)))
    for i in teams_names:
        to_win[i] = 0
        to_reach_final[i] = 0
        to_reach_semis[i] = 0
        to_reach_qurter[i] = 0
        to_qualify_group[i] = 0
        
        position_1[i] = 0
        position_2[i] = 0
        position_3[i] = 0
        position_4[i] = 0
        points[i] = 0
        goals_group[i] = 0
    
    for a in range(0,N_sim):
        group_summary, df_tie_break_g_tot = Group_Sim_2(df =df)
        group_summary['Rank'] = np.zeros(len(group_summary))

        for i in Groups:
            rank = group_first(bb= group_summary[group_summary['Group']==i],tie_break = df_tie_break_g_tot)
            k = 4
            for j in rank:
                group_summary.loc[group_summary['Team'] == j, 'Rank'] = k 
                k -= 1
    
        knock_out = list(group_summary[(group_summary['Rank'] != 3)*(group_summary['Rank'] != 4)].sort_values(['Group','Rank'], ascending=True)['Team'])
        best_third_table = group_summary[group_summary['Rank'] == 3].sort_values(['Points','Goals_diff','Goals'])
        best_third_table = best_third_table.iloc[2:]
        best_third_table = best_third_table.sort_values(['Group'])
        
        for i in teams_names:
            points[i] += group_summary[group_summary['Team'] == i]['Points'].values[0]
            goals_group[i] += group_summary[group_summary['Team'] == i]['Goals'].values[0]
            
        for i in list(group_summary[group_summary['Rank'] == 1]['Team'].values):
            position_1[i] += 1
            
        for i in list(group_summary[group_summary['Rank'] == 2]['Team'].values):
            position_2[i] += 1
            
        for i in list(group_summary[group_summary['Rank'] == 3]['Team'].values):
            position_3[i] += 1
            
        for i in list(group_summary[group_summary['Rank'] == 4]['Team'].values):
            position_4[i] += 1

    
        winner,final,semis,quarter_f,knock_out_seed = knock_out_phase(knock_out,best_third_table,df = df)
        to_win[winner] += 1
        to_reach_final[final[0]] += 1 
        to_reach_final[final[1]] += 1 
        for i in semis:
            to_reach_semis[i] += 1
            
        for i in quarter_f:
            to_reach_qurter[i] += 1
        
        for i in knock_out_seed:
            to_qualify_group[i] += 1
            
    
    a = pd.DataFrame(list(points.items()), columns=['Team', 'Points'])
    b = pd.DataFrame(list(position_1.items()), columns=['Team', 'First'])
    c = pd.DataFrame(list(position_2.items()), columns=['Team', 'Second'])
    d = pd.DataFrame(list(position_3.items()), columns=['Team', 'Third'])
    e = pd.DataFrame(list(position_4.items()), columns=['Team', 'Last'])
    f = pd.DataFrame(list(goals_group.items()), columns=['Team', 'Goals_Group_Stage'])

    s = pd.merge(a,b, on='Team')
    s = pd.merge(s,c, on='Team')
    s = pd.merge(s,d, on='Team')
    s = pd.merge(s,e, on='Team')
    group_market = pd.merge(s,f, on='Team') 
    group_market = pd.merge(group_market,df, on='Team') 
    
    num_columns = ['First','Second','Third','Last','Goals_Group_Stage','Points']
    group_market[num_columns] = group_market[num_columns].div(N_sim, axis=0)
    
    a = pd.DataFrame(list(to_win.items()), columns=['Team', 'Winner'])
    b = pd.DataFrame(list(to_reach_final.items()), columns=['Team', 'Finalist'])
    c = pd.DataFrame(list(to_reach_semis.items()), columns=['Team', 'Semi_Final'])
    d = pd.DataFrame(list(to_reach_qurter.items()), columns=['Team', 'Quarter_Final'])
    e = pd.DataFrame(list(to_qualify_group.items()), columns=['Team', 'Qulify_Group_Stage'])
    f = pd.DataFrame(list(to_qualify_group.items()), columns=['Team', 'Qulify_Group_Stage'])

    s = pd.merge(a,b, on='Team')
    s = pd.merge(s,c, on='Team')
    s = pd.merge(s,d, on='Team')
    outright = pd.merge(s,e, on='Team')

    num_columns = [ 'Winner','Finalist', 'Semi_Final', 'Quarter_Final', 'Qulify_Group_Stage']
    outright[num_columns] = outright[num_columns].div(N_sim, axis=0)
    
    return outright,group_market

outright, group_market = All_Sim()


In [93]:
group_market.sort_values(['Group','Points'])

,Team,Points,First,Second,Third,Last,Goals_Group_Stage,Group,Ratings
1,Scotland,2.909,0.097,0.191,0.305,0.407,3.121,A,-1.333944
2,Hungary,3.601,0.137,0.279,0.296,0.288,3.629,A,-1.166863
3,Switzerland,3.821,0.153,0.305,0.288,0.254,3.624,A,-1.055169
0,Germany,6.225,0.613,0.225,0.111,0.051,5.857,A,-0.115000
7,Albania,1.323,0.019,0.051,0.169,0.761,2.398,B,-1.782317
6,Italy,4.972,0.296,0.318,0.304,0.082,5.077,B,-0.220000
5,Croatia,5.100,0.320,0.314,0.289,0.077,5.047,B,-0.190000
4,Spain,5.329,0.365,0.317,0.238,0.080,5.221,B,-0.096620
8,Slovenia,2.809,0.062,0.205,0.319,0.414,3.162,C,-1.508252
10,Serbia,3.346,0.123,0.248,0.307,0.322,3.546,C,-1.227027


In [94]:
outright.sort_values('Winner',ascending = False)

,Team,Winner,Finalist,Semi_Final,Quarter_Final,Qulify_Group_Stage
15,France,0.135,0.247,0.403,0.635,0.896
22,Portugal,0.120,0.227,0.388,0.664,0.966
11,England,0.114,0.216,0.381,0.656,0.941
13,Netherlands,0.113,0.223,0.390,0.626,0.881
0,Germany,0.105,0.183,0.337,0.602,0.935
4,Spain,0.101,0.177,0.348,0.577,0.870
16,Belgium,0.089,0.198,0.336,0.584,0.957
5,Croatia,0.088,0.172,0.334,0.571,0.869
6,Italy,0.074,0.146,0.300,0.546,0.858
9,Denmark,0.011,0.023,0.086,0.302,0.697


Below an alternative simulation is presented with updated Group stage results. The code in the next cell is creating a csv fine with all possible games, then you can manually add the observed scores of each event.

In [23]:
#fixtures_all_home = []
#fixtures_all_away = []
#fixtures_all_group = []
#Groups = np.unique(df["Group"])
#for i in Groups:   
#    teams_names = df[df["Group"]==i]["Team"]
#    games = list(itertools.combinations(teams_names, 2))
#    for j in games:
#        fixtures_all_home.append(j[0])
#        fixtures_all_away.append(j[1])
#        fixtures_all_group.append(i)
        
#df_fixtures = pd.DataFrame({'Home' : fixtures_all_home, 'Away':fixtures_all_away, 'Group' : fixtures_all_group,
#        'Home_Goals': -1*np.ones(len(fixtures_all_home)),'Away_Goals': -1*np.ones(len(fixtures_all_home))})

#df_fixtures.to_csv('fixtures.csv', encoding='utf-8', index=False)

Below the results inserted are presented (start of 25/06/2024).

In [96]:
df_fixtures = pd.read_csv('fixtures.csv')
df_fixtures

,Home,Away,Group,Home_Goals,Away_Goals
0,Germany,Scotland,A,5,1
1,Germany,Hungary,A,2,0
2,Germany,Switzerland,A,1,1
3,Scotland,Hungary,A,0,1
4,Scotland,Switzerland,A,1,1
5,Hungary,Switzerland,A,1,3
6,Spain,Croatia,B,3,0
7,Spain,Italy,B,1,0
8,Spain,Albania,B,1,0
9,Croatia,Italy,B,1,1


In [97]:
def Group_Sim_2(df = df, df_fixtures = df_fixtures):
    
    '''This function simulates the group stage of the european cup based on a given table, which includes teams'
        names and their group and their power ratings. It returns a dataframe icluding teams' points, goals and
        goals' difference for each team in the group stage and another dataframe including all the simulated scores
        in order to be used for tie break cases.'''
    
    array = np.zeros((len(df),len(df))) 
    index_values = df['Team']
    column_values = df['Team']
    df_tie_break_g_tot = pd.DataFrame(data = array, index = index_values, columns = column_values) 
    
    points = {}
    team_H_group_goals = {}
    team_diff_group_goals = {}
    points_single_sim = {}
    goals_h_single_sim = {}
    goals_a_single_sim = {}
    goals_h_diff_single_sim = {}
    goals_a_diff_single_sim = {}
    teams = df['Team']
    #fix_ex = fix.copy()
    
    for i in range(0,len(teams)):
        points[teams[i]] = 0
        team_H_group_goals[teams[i]] = 0
        team_diff_group_goals[teams[i]] = 0
    
    Groups = np.unique(df["Group"])
    for i in Groups:   
        teams_names = df[df["Group"]==i]["Team"]
        games = list(itertools.combinations(teams_names, 2))
        for j in teams_names:
            points_single_sim[j] = 0
            goals_h_single_sim[j] = 0
            goals_a_single_sim[j] = 0
            goals_h_diff_single_sim[j] = 0
            goals_a_diff_single_sim[j] = 0
        
        for j in games:
            
            df_actual = df_fixtures[(df_fixtures["Home"]==j[0])*(df_fixtures["Away"]==j[1])].iloc[0]
            
            actual_resu_home = df_actual['Home_Goals']
            actual_resu_away = df_actual['Away_Goals']
            if actual_resu_home !=-1:
                home_G = actual_resu_home
                away_G =  actual_resu_away
            
            else:
                team_A_rating = df[df["Team"]==j[0]]["Ratings"].values[0]
                team_B_rating = df[df["Team"]==j[1]]["Ratings"].values[0]
            
                home_G, away_G = game_result(team_A_rating,team_B_rating)
            
            
            if home_G > away_G :
                points_single_sim[j[0]] += 3
            elif home_G < away_G :
                points_single_sim[j[1]] += 3
            else:
                points_single_sim[j[0]] += 1
                points_single_sim[j[1]] += 1
            
            df_tie_break_g_tot[j[0]][j[1]] = home_G
            df_tie_break_g_tot[j[1]][j[0]] = away_G
                
            goals_h_single_sim[j[0]] += home_G
            goals_a_single_sim[j[1]] += away_G
            goals_h_diff_single_sim[j[0]] += home_G - away_G
            goals_a_diff_single_sim[j[1]] += away_G - home_G
        
        for i in teams_names:
            points[i] = points_single_sim[i]
            team_H_group_goals[i] = goals_h_single_sim[i]+goals_a_single_sim[i]
            team_diff_group_goals[i] = goals_h_diff_single_sim[i]+goals_a_diff_single_sim[i]
    

    
    a = pd.DataFrame(list(points.items()), columns=['Team', 'Points'])
    b = pd.DataFrame(list(team_H_group_goals.items()), columns=['Team', 'Goals'])
    c = pd.DataFrame(list(team_diff_group_goals.items()), columns=['Team', 'Goals_diff'])
    
    group_summary = pd.merge(a,b, on='Team')
    group_summary = pd.merge(group_summary,c, on='Team')
    group_summary = pd.merge(group_summary,df[['Team','Group']],on='Team')
    
    
    return group_summary, df_tie_break_g_tot

a,b = Group_Sim_2()

These are the updated odds:

In [98]:
outright, group_market = All_Sim()
group_market.sort_values(['Group','Points'])

,Team,Points,First,Second,Third,Last,Goals_Group_Stage,Group,Ratings
1,Scotland,1.000,0.000,0.000,0.000,1.000,2.000,A,-1.333944
2,Hungary,3.000,0.000,0.000,1.000,0.000,2.000,A,-1.166863
3,Switzerland,5.000,0.000,1.000,0.000,0.000,5.000,A,-1.055169
0,Germany,7.000,1.000,0.000,0.000,0.000,8.000,A,-0.115000
7,Albania,1.000,0.000,0.000,0.000,1.000,3.000,B,-1.782317
5,Croatia,2.000,0.000,0.000,1.000,0.000,3.000,B,-0.190000
6,Italy,4.000,0.000,1.000,0.000,0.000,3.000,B,-0.220000
4,Spain,9.000,1.000,0.000,0.000,0.000,5.000,B,-0.096620
10,Serbia,2.237,0.000,0.292,0.126,0.582,2.270,C,-1.227027
8,Slovenia,2.521,0.079,0.050,0.646,0.225,2.840,C,-1.508252


In [99]:
outright.sort_values('Winner',ascending = False)

,Team,Winner,Finalist,Semi_Final,Quarter_Final,Qulify_Group_Stage
15,France,0.169,0.305,0.494,0.747,1.000
11,England,0.136,0.235,0.437,0.724,1.000
22,Portugal,0.115,0.202,0.391,0.727,1.000
4,Spain,0.105,0.206,0.420,0.760,1.000
16,Belgium,0.103,0.176,0.290,0.525,0.893
6,Italy,0.102,0.213,0.371,0.694,1.000
0,Germany,0.101,0.196,0.380,0.709,1.000
13,Netherlands,0.099,0.240,0.442,0.706,1.000
14,Austria,0.018,0.050,0.140,0.414,0.995
3,Switzerland,0.014,0.040,0.115,0.306,1.000
